In [12]:
import os
from operator import itemgetter
from typing import List, Tuple

from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, format_document
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import (
    RunnableBranch,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)
import pinecone # noqa
from langchain.vectorstores import Pinecone
from pydantic import BaseModel, Field

In [13]:
load_dotenv()

True

In [14]:
api_pinecone = os.getenv('PINECONE_API_KEY')
env_pinecone = os.getenv('PINECONE_ENV')

pinecone.init(
    api_key=api_pinecone,
    environment=env_pinecone
)
index_name = "constitution-idx"

In [23]:
vectorstore = Pinecone.from_existing_index(index_name, OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [24]:
_template = """
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """
Dado el contexto proporcionado, debes responder asumiendo el rol de un experto en derecho constitucional con un inmenso conocimiento y experiencia en el campo.
Debes responder las preguntas basado en tu conocimiento y en la conversación previa. no inventes respuestas.
Reformula la pregunta para que sea una pregunta independiente, en su idioma original y mantenga el contexto de la conversación.
En el contexto encontrarás tres documentos los cuales empiezan con inicio del documento seguido del titulo y finalizan con fin del documento.

los documentos hacen referencia a la constitución actual de la república de chile, la propuestas para el cambio de la constitución del 2022 y la propuesta para el cambio de la constitución del 2023.

En el contexto encontrarás tres documentos los cuales empiezan con INICIO DEL DOCUMENTO seguido del TITULO y finalizan con FIN DEL DOCUMENTO.
El primer documento es la Constitución actual de la República de chile
El segundo documento es la propuestas para el cambio de la constitución del 2022 y tiene como titulo propuesta de nueva constitución 2022
El tercer documento es la propuesta para el cambio de la constitución del 2023 y tiene como titulo propuesta de nueva constitución 2023
        
<context>
{context}
</context>"""

ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{question}"),
    ]
)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

In [25]:
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [28]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [29]:
class ChatHistory(BaseModel):
    chat_history: List[Tuple[str, str]] = Field(..., extra={"widget": {"type": "chat"}})
    question: str

In [30]:
_search_query = RunnableBranch(
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    RunnableLambda(itemgetter("question")),
)

_inputs = RunnableMap(
    {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "context": _search_query | retriever | _combine_documents,
    }
).with_types(input_type=ChatHistory)

chain = _inputs | ANSWER_PROMPT | ChatOpenAI() | StrOutputParser()

In [37]:
question = "Cuantas propuestas de nueva constitución tienes ?"
answer = chain.invoke(
    {
        "question": question,
        "chat_history": [],
    }
)
answer

'Según el contexto proporcionado, hay dos propuestas de nueva constitución:\n1. "Propuesta de nueva constitución 2022".\n2. "Propuesta de nueva constitución 2023".'

In [38]:
chat_history = [(question, answer)]
question = "En la constitución actual o vigente de chile, que aborda sobre temas de educación"
answer = chain.invoke(
    {
        "question": question,
        "chat_history": [],
    }
)

answer

'En la Constitución actual de la República de Chile, se abordan diversos temas relacionados con la educación. A continuación, destacaré algunos de los artículos pertinentes:\n\n- Artículo 19, número 10: Este artículo establece que el Estado tiene el deber de proporcionar educación pública, pluralista y de calidad en todos los niveles. Además, se garantiza el financiamiento de los establecimientos de educación parvularia, básica y media.\n\n- Artículo 19, número 11: En este artículo, se establece el deber de la familia y la comunidad de contribuir al desarrollo y perfeccionamiento de la educación. Asimismo, se establece que corresponde al Estado asegurar la calidad de la educación en todos sus niveles, fomentar la formación cívica, estimular la investigación científica y tecnológica, la creación artística y la protección del patrimonio cultural de la Nación.\n\n- Artículo 19, número 24: Este artículo consagra la libertad de enseñanza, reconociendo el derecho de los padres a elegir la ed

In [39]:
chat_history = [(question, answer)]
answer = chain.invoke(
    {
        "question": "y en comparación con la propuesta del 2023, analiza cuales son las principales diferencias en temas de educación",
        "chat_history": chat_history,
    }
)
answer

'La propuesta de nueva Constitución del 2023 aún no ha sido presentada en su totalidad, por lo que no puedo proporcionar una comparación detallada de las diferencias en temas de educación entre la Constitución actual y la propuesta del 2023.\n\nSin embargo, con base en la información proporcionada en el fragmento del documento de la propuesta del 2023, se puede observar que se mantienen algunos aspectos fundamentales, como el deber del Estado de proporcionar educación pública, pluralista y de calidad en todos los niveles. También se enfatiza en la importancia de asegurar la calidad de la educación, fomentar la formación cívica y promover el desarrollo profesional y respeto de los docentes.\n\nEs importante tener en cuenta que la propuesta completa de la nueva Constitución del 2023 deberá ser analizada en su totalidad para comprender las posibles diferencias en temas de educación con respecto a la Constitución actual.'

In [40]:
chat_history = [(question, answer)]
answer = chain.invoke(
    {
        "question": "y en comparación con la propuesta del 2022, analiza cuales son las principales diferencias en temas de educación entre esta propuesta (2022) y la del 2023",
        "chat_history": chat_history,
    }
)
answer

'Lamentablemente, no tengo acceso al contenido completo de las propuestas de nueva Constitución del 2022 y del 2023. Solo se proporcionó un fragmento del documento de la propuesta del 2022 y no se ha proporcionado ninguna información sobre la propuesta del 2023 más allá de su título.\n\nBasándome en la información limitada proporcionada, no puedo realizar una comparación detallada de las diferencias en temas de educación entre la propuesta del 2022 y la del 2023.\n\nPara una evaluación precisa de las diferencias en temas de educación entre ambas propuestas, es necesario tener acceso al contenido completo de ambas propuestas y analizarlas en su totalidad. Sin esa información, no puedo proporcionar una respuesta precisa a tu pregunta.'